In [1]:
# check versions and devices
import tensorflow as tf
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
print("python version: ", sys.version)
# print("tf version: ", tf.__version__)
# print("devices: ", tf.config.list_physical_devices())

2022-11-15 18:10:12.573581: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


python version:  3.9.13 (main, Oct 13 2022, 16:12:30) 
[Clang 12.0.0 ]


In [2]:
# create train and validation datasets
batch_size = 126
img_height = 224
img_width = 224

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "./data/images/",
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=True
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "./data/images/",
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=True
)

train_ds.class_names

Found 16041 files belonging to 2 classes.
Using 12833 files for training.
Metal device set to: AMD Radeon Pro 560X

systemMemory: 32.00 GB
maxCacheSize: 2.00 GB

Found 16041 files belonging to 2 classes.
Using 3208 files for validation.


['car', 'not_car']

In [3]:
# configure performance
autotune = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=autotune)
val_ds = val_ds.cache().prefetch(buffer_size=autotune)

In [4]:
# transfer lerning from tensorflow hub
import tensorflow_hub as hub
resnet_50 = "https://tfhub.dev/tensorflow/resnet_50/classification/1"

In [5]:
# model
model = tf.keras.Sequential([
    # input layer
    tf.keras.layers.InputLayer(input_shape=(img_height, img_width, 3)),
    # augmentations
    #tf.keras.layers.RandomFlip('horizontal'),
    #tf.keras.layers.RandomRotation(0.2),
    # rescaling
    tf.keras.layers.Rescaling(1./255),
    # transfer learning
    hub.KerasLayer(resnet_50, trainable=False, dtype=tf.float32),
    # dropout
    tf.keras.layers.Dropout(0.2),
    # output layer
    tf.keras.layers.Dense(1, activation="softmax")
])
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 keras_layer (KerasLayer)    (None, 1001)              25612201  
                                                                 
 dropout (Dropout)           (None, 1001)              0         
                                                                 
 dense (Dense)               (None, 1)                 1002      
                                                                 
Total params: 25,613,203
Trainable params: 1,002
Non-trainable params: 25,612,201
_________________________________________________________________


In [6]:
# compile model
model.compile(
    optimizer='adam',
    loss=tf.losses.BinaryCrossentropy(),
    metrics=['accuracy']
)

In [7]:
# train
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=3
)

Epoch 1/3
  4/102 [>.............................] - ETA: 5:13 - loss: 0.6905 - accuracy: 0.5397

KeyboardInterrupt: 

In [ ]:
# save and load
import pandas as pd
from tensorflow.keras.models import load_model

# save
model.save('model.h5')
hist_df = pd.DataFrame(history.history)
with open("hist.json", "w") as f:
    hist_df.to_json(f)


In [ ]:
# load
model = load_model('model.h5',custom_objects={'KerasLayer':hub.KerasLayer})
model_hist = pd.read_json("hist.json")

In [ ]:
# plot accuracy and val_accuracy
import matplotlib.pyplot as plt
plt.plot(model_hist['accuracy'], label='train')
plt.plot(model_hist['val_accuracy'], label = 'test')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='upper left')
plt.show()

In [ ]:
# plot loss and val_loss
plt.plot(model_hist['loss'], label='train')
plt.plot(model_hist['val_loss'], label = 'test')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper left')
plt.show()